# 数据规整化：清理、转换、合并、重塑

数据**分析和建模**方面的大量编程工作都是用在数据准备上的:**加载**、**清理**、**转换**以及**重塑**。有时候,存放在文件或数据库中的数据并不能满足你的数据处理应用的要求。许多人都选择使用通用编程语言(如Python、Perl、R或Java)或UNIX文本处理工具(如sed或awk)对数据格式进行专门处理。幸运的是,pandas和Python标准库提供了一组**高级**的、**灵活**的、**高效**的**核心函数**和**算法**,它们使你能够轻松地将数据规整化为正确的形式。

In [213]:
import pandas as pd
from pandas import Series
from pandas import DataFrame

import numpy as np

import json

## 合并数据集

pandas对象中的数据可以通过一些**内置**的方式进行合并：
* pandas.merge可根据**一个或多个键**将不同DataFrame中的**行**连接起来，实现的就是数据库的连接操作 -- **横向合并**；
* pandas.concat可以沿着一条**轴**将多个对象**堆叠**到一起 -- **纵向合并**；
* 实例方法combine_first可以将重复数据编接在一起,用一个对象中的值**填充**另一个对象中的**缺失值** -- **填充式**；

### 数据库风格的DataFrame合并 -- pandas.merge

#### 普通多对一默认列默认合并方式合并

In [2]:
df1 = DataFrame({'key':['a','b','c','a','b','c'],
                'data1':range(6)})
df2 = DataFrame({'key':['a','d'],
                'data2':range(2)})
pd.merge(df1,df2)

,data1,key,data2
0,0,a,0
1,3,a,0


#### 索引上的合并 -- left_index、right_index

简单索引情况下：

In [3]:
df1 = DataFrame({'key':['a','b','c','a','a'],
                'data':range(5)})
df2 = DataFrame({'data2':range(3)},index=['a','b','a'])
pd.merge(df1, df2, left_on='key', right_index=True, how='outer') # 需要同时指定右侧使用索引作为连接键，同时指定左侧的连接键列名

,data,key,data2
0,0,a,0.0
0,0,a,2.0
3,3,a,0.0
3,3,a,2.0
4,4,a,0.0
4,4,a,2.0
1,1,b,1.0
2,2,c,NaN


两侧都是索引的情况下：

In [4]:
df1 = DataFrame(np.arange(4), index=['a','b','a','c'])
df2 = DataFrame(np.arange(4), index=['a','b','b','d'])
pd.merge(df1, df2, left_index=True, right_index=True, suffixes=['_df1','_df2'])

,0_df1,0_df2
a,0,0
a,2,0
b,1,1
b,1,2


层次化索引情况下（此时需要显示指明用于连接索引的多个列）：

In [5]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
# 其中righth是一个层次化索引的DataFrame，直接指定right_index=True试试看，但是此时需要指定left_on使用多个键来连接
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


#### merge函数参数描述

* left：用于合并的左侧DataFrame；
* right：用于合并的右侧DataFrame；
* how：合并方式，“inner”（保留交集）、“outer”（保留并集）、“left”（保留左侧）、“right”（保留右侧），默认为inner；
* on：用于连接的列名，必须同时存在于左右两侧，如果本身以及left_on、right_on都未指定，则使用左右两侧的列名交集来连接；
* left_on：左侧DataFrame中用于连接的列名；
* right_on：右侧DataFrame中用于连接的列名；
* left_index：将左侧行索引用作其连接键，设置为True表示左侧索引也可以作为连接键；
* right_index：将右侧行索引用作其连接键，设置为True表示右侧索引也可以作为连接键；
* sort：根据连接的键对数据进行排序，默认是True，但是在处理大数据时，设置为False可以得到更好的性能；
* suffixes：字符串值元组，用于追加到重叠的列名后，默认是“_x”、“_y”，如果有列data重复，那么合并后会得到data_x、data_y；
* copy：是否复制到结果数据结构中，默认为True；

**警告**: 在进行列-列连接时,DataFrame对象中的索引会被丢弃；

### DataFrame.join方法实现索引上的合并

#### 左右两侧均使用索引进行连接

In [6]:
df1 = DataFrame(np.arange(4), index=['a','b','a','c'])
df2 = DataFrame(np.arange(4), index=['a','b','b','d'])
# 默认how是left，使用两侧的索引来连接，在有重复列名时必须指定lsuffix以及rsuffix
df1.join(df2, lsuffix='_left', rsuffix='_right', how='left') 

,0_left,0_right
a,0,0.0
a,2,0.0
b,1,1.0
b,1,2.0
c,3,NaN


#### 左侧使用列作为索引进行连接

In [7]:
df1 = DataFrame({'key':['a','b','c','a','a'],
                'data':range(5)})
df2 = DataFrame({'data2':range(3)},index=['a','b','a'])
# 可以指定左侧的列作为索引连接右侧的索引
df1.join(df2, on='key', how='inner')

,data,key,data2
0,0,a,0
0,0,a,2
3,3,a,0
3,3,a,2
4,4,a,0
4,4,a,2
1,1,b,1


#### 多个DataFrame的简单连接

In [8]:
df1 = DataFrame(np.arange(4), index=['a','b','a','c'])
df2 = DataFrame(np.arange(4), index=['a','b','b','d'])
df3 = DataFrame(np.arange(4), index=['a','b','d','e'])
df4 = DataFrame(np.arange(4), index=['a','b','d','e'])
df1.join([df2,df3,df4], lsuffix=['1','2'], rsuffix=['3','4']) # 多个DataFrame时不起作用了

,0_x,0_y,0_x,0_y
a,0.0,0.0,0.0,0.0
a,2.0,0.0,0.0,0.0
b,1.0,1.0,1.0,1.0
b,1.0,2.0,1.0,1.0
c,3.0,NaN,NaN,NaN
d,NaN,3.0,2.0,2.0
e,NaN,NaN,3.0,3.0


### 轴向连接

另一种数据合并运算也被称作**连接**(concatenation)、**绑定**(binding)或**堆叠**(stacking)；

#### NumPy合并原始数据 -- concatenate

In [9]:
arr = np.arange(16).reshape(4,4)
print np.concatenate([arr,arr], axis=1) # axis为1的合并，也就是横向合并，类似上面的DataFrame的合并
print np.concatenate([arr,arr]) # axis为0的合并，纵向合并，说是堆叠，连接更贴切

[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]
 [12 13 14 15 12 13 14 15]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


#### pandas.concat简单连接没有重复索引的三个series，参数全默认

In [10]:
se1 = Series([1,3,5], index=['a','b','c'])
se2 = Series([4,6], index=['d','e'])
se3 = Series([7,8], index=['f','g'])
pd.concat([se1,se2,se3]) # 默认是纵向连接的，也就是堆叠式

a    1
b    3
c    5
d    4
e    6
f    7
g    8
dtype: int64

#### pandas.concat -- 指定参数axis=1，使其横向合并

In [11]:
pd.concat([se1,se2,se3], axis=1) # 能够看到此时出现大量的NaN

,0,1,2
a,1.0,NaN,NaN
b,3.0,NaN,NaN
c,5.0,NaN,NaN
d,NaN,4.0,NaN
e,NaN,6.0,NaN
f,NaN,NaN,7.0
g,NaN,NaN,8.0


#### pandas.concat -- 指定参数join，设置连接方式

In [12]:
pd.concat([se1,se2,se3], join='inner') # 能够看到此时出现大量的NaN

a    1
b    3
c    5
d    4
e    6
f    7
g    8
dtype: int64

#### pandas.concat -- 指定参数join_axes，指定要在其他轴上使用的索引

In [13]:
pd.concat([se1,se2,se3], axis=1, join_axes=[['a','g','e']]) # 注意此处join_axes的写法，是list的list

,0,1,2
a,1.0,NaN,NaN
g,NaN,NaN,8.0
e,NaN,6.0,NaN


#### pandas.concat -- 指定参数keys，创建层次化索引

In [14]:
pd.concat([se1,se1], keys=['first', 'second', 'third'])

first   a    1
        b    3
        c    5
second  a    1
        b    3
        c    5
dtype: int64

In [15]:
# 指定axis为1时，keys作为列名，这个相当于先按axis为0进行concat，然后再unstack转换到DataFrame
pd.concat([se1,se1], axis=1, keys=['first', 'second', 'third'])

,first,second
a,1,1
b,3,3
c,5,5


#### pandas.concat -- 针对DataFrame

In [16]:
df1 = DataFrame(np.arange(16).reshape(4,4), 
                columns=['a','b','c','d'],
               index=['mary','lily','bench','mark'])
df2 = DataFrame(np.arange(16).reshape(4,4), 
                columns=['e','f','g','h'],
               index=['mark','lily','lily','sary'])
print df1
print df2
pd.concat([df1,df2], keys=['level_1','level_2'])

        a   b   c   d
mary    0   1   2   3
lily    4   5   6   7
bench   8   9  10  11
mark   12  13  14  15
       e   f   g   h
mark   0   1   2   3
lily   4   5   6   7
lily   8   9  10  11
sary  12  13  14  15


a     b     c     d     e     f     g     h
level_1 mary    0.0   1.0   2.0   3.0   NaN   NaN   NaN   NaN
        lily    4.0   5.0   6.0   7.0   NaN   NaN   NaN   NaN
        bench   8.0   9.0  10.0  11.0   NaN   NaN   NaN   NaN
        mark   12.0  13.0  14.0  15.0   NaN   NaN   NaN   NaN
level_2 mark    NaN   NaN   NaN   NaN   0.0   1.0   2.0   3.0
        lily    NaN   NaN   NaN   NaN   4.0   5.0   6.0   7.0
        lily    NaN   NaN   NaN   NaN   8.0   9.0  10.0  11.0
        sary    NaN   NaN   NaN   NaN  12.0  13.0  14.0  15.0

#### pandas.concat -- ignore_index忽略跟分析无关的索引

In [17]:
df1 = DataFrame(np.arange(16).reshape(4,4), columns=['a','b','c','d'])
df2 = DataFrame(np.arange(9).reshape(3,3), columns=['a','b','c'])
pd.concat([df1,df2], ignore_index=True) # igonre_index设置为True后，索引会被重新默认值填充掉

,a,b,c,d
0,0,1,2,3.0
1,4,5,6,7.0
2,8,9,10,11.0
3,12,13,14,15.0
4,0,1,2,NaN
5,3,4,5,NaN
6,6,7,8,NaN


#### concat参数

* objs：参与连接的列表或者字典，唯一必填参数；
* axis：连接的轴向，默认为0，纵向；
* join：“inner”交集/“outer”并集，默认是“outer”；
* join_axes：指明用于其他n-1条轴的索引，不执行并集/交集运算；
* keys：与连接对象有关的值，用于形成向上的层次化索引，可以是任意值的列表或数组、元组数组、数组列表（如果将levels设置为多级数组的话）
* levels：指定用作层次化索引各级别上的索引，如果设置了keys的话；
* names：用于创建分层级别的名称，如果设置了keys和levels的话；
* verify_integrity：检查结果对象新轴上的重复情况，发现则报异常，默认为False，即允许重复；
* ignore_index：不保留连接轴上的索引，产生一组新索引range（total_length）；

### 合并重叠数据 -- 用一组数据根据某种条件填充另一组数据

还有一种数据组合问题不能用简单的合并(merge)或连接(concatenation)运算来处理，比如说,你可能有索引全部或部分重叠的两个数据集；

#### 例子：NumPy的where函数,它用于表达一种矢量化的if-else

In [18]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print a
print '\n'
print b
print '\n'
# 生成一个布尔型数组用于标示对应位置是从b中获取，还是a，表达如果a中某位置是null，那么该位置的值从b中获取，否则从a中获取
np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64


f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64




array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

#### Series.combine_first方法 -- 一样实现上述功能，同时还会进行数据对齐

In [19]:
b[:-2].combine_first(a[2:]) # 取b中的0到倒数第2个，以及a中第二个到最后一个，出现重叠优先取b中的生成结果

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

#### DataFrame.combine_first方法

In [20]:
df1 = DataFrame(np.arange(16).reshape(4,4))
df2 = DataFrame(np.arange(9).reshape(3,3))
df1.ix[1:2,1:2] = np.nan
print df1
print '\n'
print df2
print '\n'
df1.combine_first(df2) # 默认使用同样位置的元素去填充df1中的NaN值

    0     1     2   3
0   0   1.0   2.0   3
1   4   NaN   NaN   7
2   8   NaN   NaN  11
3  12  13.0  14.0  15


   0  1  2
0  0  1  2
1  3  4  5
2  6  7  8




/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3
0,0,1.0,2.0,3
1,4,4.0,5.0,7
2,8,7.0,8.0,11
3,12,13.0,14.0,15


## 重塑和轴向旋转

有许多用于重新排列表格型数据的基础运算。这些函数也称作**重塑**(reshape)或**轴向旋转**(pivot)运算；

### 重塑层次化索引

#### stack -- 将数据的列旋转为行

In [21]:
df = DataFrame(np.arange(4).reshape(2,2),
              columns=['a','b'],
              index=['first','second'])
df.stack() # 原来的columns转换成了内层的索引，DataFrame转成了Series

first   a    0
        b    1
second  a    2
        b    3
dtype: int64

#### unstack -- 将数据的行旋转为列

In [22]:
se = Series(np.arange(4), index=[['a','a','b','b'],['first','second','first','second']])
se.unstack() # 原来的内层索引变成了列名

,first,second
a,0,1
b,2,3


#### 默认unstack、stack都是操作的**最内层**索引，可以通过指定索引号或索引名来指定

In [23]:
se.unstack(0) # 默认总是n-1，指定0操作最外层索引

,a,b
first,0,2
second,1,3


#### 对DataFrame进行stack、unstack时旋转后的轴总是在最内层

In [24]:
df = DataFrame(np.arange(18).reshape(6,3),
              columns=['math','english','chinese'],
              index=[['mary','mary','john','john','lily','lily'],['middle','final','middle','final','middle','final']])

In [25]:
df.unstack() # 将索引转为层次最内层的列，此处使用默认最内层索引

math        english        chinese       
     final middle   final middle   final middle
john     9      6      10      7      11      8
lily    15     12      16     13      17     14
mary     3      0       4      1       5      2

In [26]:
df.unstack().stack(0) # 将原列转为最内层索引

final  middle
john chinese     11       8
     english     10       7
     math         9       6
lily chinese     17      14
     english     16      13
     math        15      12
mary chinese      5       2
     english      4       1
     math         3       0

### 将“长格式”转换为“宽格式” -- 将某个列的全部取值转换成对应的一个一个的列存在，增加列，减少行，所以是长->宽

比如数据库中有一列叫课程，那么就会出现以下这几条记录的情况：
* helong,math,80
* helong,chinese,85
* helong,english,55

这样的设计有好有坏：
* 好处：
    * 课程的取值可以随意扩展；
* 坏处：
    * 数据操作复杂；
    * 数据冗余度高，比如helong由于在三行记录中都有，因此会重复N次，N等于课程的取值数；
    
此处姓名的表示方式就称之为“长格式”，而所谓的转换为“宽格式”如下：

        math,chinese,englisth
    name
        helong,80,85,55

能够看到原来的三行数据变为了一行（长到短），而一行三个数据变为四个（窄变宽）；

#### 将lesson转换到列

In [27]:
df = DataFrame({
    'name':['mary','mary','john','john','lily','lily'],
    'lesson':['chinese','english','chinese','english','chinese','english'],
    'grade':[55,66,77,44,33,88]
})
df

,grade,lesson,name
0,55,chinese,mary
1,66,english,mary
2,77,chinese,john
3,44,english,john
4,33,chinese,lily
5,88,english,lily


In [28]:
df.pivot('name','lesson','grade')

lesson,chinese,english
name,,
john,77,44
lily,33,88
mary,55,66


#### 将lesson转换到列 -- 多数据列

In [29]:
df = DataFrame({
    'name':['mary','mary','john','john','lily','lily'],
    'lesson':['chinese','english','chinese','english','chinese','english'],
    'grade':[55,66,77,44,33,88],
    'xxx':[1,2,3,4,5,6] # 增加的一条数据列，凑成多数据列
})
df

,grade,lesson,name,xxx
0,55,chinese,mary,1
1,66,english,mary,2
2,77,chinese,john,3
3,44,english,john,4
4,33,chinese,lily,5
5,88,english,lily,6


In [30]:
df.pivot('name','lesson') # 多数据列时，忽略第三个参数会生成层次化列，如果指定则只会保留指定的列作为数据列

grade             xxx        
lesson chinese english chinese english
name                                  
john        77      44       3       4
lily        33      88       5       6
mary        55      66       1       2

## 数据转换

### 移除重复数据

In [31]:
# DataFrame的重复行
df = DataFrame([[1,1],[1,1],[1,2]])
df

,0,1
0,1,1
1,1,1
2,1,2


#### duplicated -- 返回各行是否是重复行的Series

In [32]:
df.duplicated() # 注意：虽然第一第二行是一样的，但是这个比较是单向向前的，也就是说第一行肯定不会是重复，第二行只跟第一行比较

0    False
1     True
2    False
dtype: bool

#### drop_duplicates -- 返回移除了重复行的DataFrame

In [33]:
df.drop_duplicates() # 等价于df[~df.duplicated()]

,0,1
0,1,1
2,1,2


In [34]:
df[~df.duplicated()]

,0,1
0,1,1
2,1,2


#### 默认是判断全部列相等才算重复，也可以指定部分行的判断

In [35]:
df.duplicated([0]) # 只要第1列的值相等，即为重复

0    False
1     True
2     True
dtype: bool

#### 默认是取第一个，可以通过keep指定：first取第一个，last取最后一个，False一个都不取

In [36]:
df.drop_duplicates(keep='last')

,0,1
1,1,1
2,1,2


In [37]:
df.drop_duplicates(keep=False)

,0,1
2,1,2


### 利用函数或者映射做数据转换

#### 数据映射

在对数据集进行转换时,你可能希望根据数组、Series或DataFrame列中的值来实现该转换工作。我们来看看下面这组有关肉类的数据:

In [38]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami','corned beef', 'Bacon', 'pastrami', 'honey ham','nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [39]:
meat_to_animal = {
 'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'
} # 用于映射的数据
meat_to_animal

{'bacon': 'pig',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon',
 'pastrami': 'cow',
 'pulled pork': 'pig'}

Series的map方法可以接受一个函数或含有映射关系的字典型对象,但是这里有一个小问题,即有些肉类的首字母大写了,而另一些则没有，因此,我们还需要将各个值转换为小写：

In [40]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 替换值 -- replace

In [41]:
se = Series([55,77,80,60,44,-1,-2])
print se.replace(-1,np.nan) # 将-1替换为NaN
print se.replace([-1,-2],np.nan) # 同时替换多个值
print se.replace([55,-1,-2],[60,np.nan,np.nan]) # 目标值也是多个
print se.replace({-1:np.nan,-2:np.nan,55:60}) # 使用字典

0    55.0
1    77.0
2    80.0
3    60.0
4    44.0
5     NaN
6    -2.0
dtype: float64
0    55.0
1    77.0
2    80.0
3    60.0
4    44.0
5     NaN
6     NaN
dtype: float64
0    60.0
1    77.0
2    80.0
3    60.0
4    44.0
5     NaN
6     NaN
dtype: float64
0    60.0
1    77.0
2    80.0
3    60.0
4    44.0
5     NaN
6     NaN
dtype: float64


### 重命名轴索引

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新对象，轴还可以被就地修改，而无需新建一个数据结构；

In [42]:
df = DataFrame(np.arange(12).reshape((3, 4)),
                index=['Ohio', 'Colorado', 'New York'],
                columns=['one', 'two', 'three', 'four'])
df

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


#### 仅修改轴索引 -- index.map

In [51]:
print df.index.map(str.upper) # 轴对象同样有map方法
def add_state(name):
    return 'State '+name
print df.index.map(add_state) # 使用自定义方法
mapping = {'Ohio':'OH', 'Colorado':'CD', 'New York':'NY'}
print df.index.map(lambda name:mapping[name]) # 使用lambda+字典映射

Index([u'OHIO', u'COLORADO', u'NEW YORK'], dtype='object')
Index([u'State Ohio', u'State Colorado', u'State New York'], dtype='object')
Index([u'OH', u'CD', u'NY'], dtype='object')


#### 创建数据集的转换板 -- 期望得到一份修改了轴索引后的数据拷贝 -- DataFrame.rename

In [54]:
# 通过rename很方便的做到：1.重命名轴索引，2.获得新数据拷贝
mapping = {'one':'>1<', 'two':'>2<', 'three':'>3<', 'four':'>4<'}
df.rename(index=add_state, columns=lambda col:mapping[col])

,>1<,>2<,>3<,>4<
State Ohio,0,1,2,3
State Colorado,4,5,6,7
State New York,8,9,10,11


In [56]:
# rename可以结合字典实现部分轴索引的重命名
df.rename(index={'Ohio':'Ohio New'}, columns={'three':'three New'})

,one,two,three New,four
Ohio New,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### 离散化和面元划分

为了便于分析，连续数据常常被离散化或拆分为“面元”(bin)，例如年龄数据映射到不同的年龄段（幼儿，少年青年等等），同样的还有体重，成绩等；

这么做的目的在于：
1. 部分算法可能不支持连续数据；
2. 连续数据会导致数据过于分散，特征不明显；
3. 很多时候面元更符合业务场景下的实际意义（比如年龄段，你很难很准确的说2岁比3岁的有什么区别，但是幼儿和青年肯定有很大区别）；

#### pandas.cut -- 根据bins对数据进行划分，指定labels命名面元名称，通过right指定闭端

In [77]:
se = Series([35,22,6,15,40,66,52,40])
bins = [0,8,16,25,40,100] # 指定面元分割点的值集合
cuts = pd.cut(se,bins,labels=['幼儿','少年','青年','中年','老年']) # 返回Series对象
print cuts

0    中年
1    青年
2    幼儿
3    少年
4    中年
5    老年
6    老年
7    中年
dtype: category
Categories (5, object): [幼儿 < 少年 < 青年 < 中年 < 老年]


In [78]:
pd.cut(se,4) # 指定划分面元个数

0    (21.0, 36.0]
1    (21.0, 36.0]
2    (5.94, 21.0]
3    (5.94, 21.0]
4    (36.0, 51.0]
5    (51.0, 66.0]
6    (51.0, 66.0]
7    (36.0, 51.0]
dtype: category
Categories (4, interval[float64]): [(5.94, 21.0] < (21.0, 36.0] < (36.0, 51.0] < (51.0, 66.0]]

**注意**：当cut的bins参数指定为int时，两端会**延长**%1以包含最大最小值，而当参数为标量数组时不会做延长；

#### pandas.qcut -- 根据样本分位数进行面元划分，得到大小基本相等的面元

In [80]:
pd.value_counts(pd.cut(np.random.randn(10000), 10))

(-0.636, 0.0986]    2790
(0.0986, 0.834]     2601
(-1.371, -0.636]    1672
(0.834, 1.568]      1428
(-2.106, -1.371]     740
(1.568, 2.303]       491
(-2.841, -2.106]     155
(2.303, 3.038]        81
(-3.583, -2.841]      26
(3.038, 3.773]        16
dtype: int64

In [81]:
pd.value_counts(pd.qcut(np.random.randn(10000), 10))

(1.279, 3.675]       1000
(0.822, 1.279]       1000
(0.508, 0.822]       1000
(0.235, 0.508]       1000
(-0.0231, 0.235]     1000
(-0.258, -0.0231]    1000
(-0.543, -0.258]     1000
(-0.86, -0.543]      1000
(-1.294, -0.86]      1000
(-4.033, -1.294]     1000
dtype: int64

**结论**：能够看到在使用qcut时，得到的面元每个包含的数量都是1000，而使用cut时则**面元大小差距很大**；

### 检测和过滤异常值

异常值(outlier)的过滤或变换运算在很大程度上其实就是数组运算；

In [112]:
np.random.seed(12345)
df = DataFrame(np.random.randn(1000, 4))
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


#### 某列中数据绝对值大于3的值

In [113]:
df[0][np.abs(df[2])>3]

5     -0.539741
102   -0.655054
324    0.050188
499   -0.293333
586    0.275144
Name: 0, dtype: float64

#### 全部值绝对值大于3的行

In [114]:
df[(np.abs(df)>3).any(axis=1)] # bool型数组还能这么用的哈！！！

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [117]:
df[(np.abs(df)>3)] = np.sign(df)*3 # 直接修改满足要求的值（实际业务中可能就是在重置异常值）
df[(np.abs(df)==3).any(axis=1)]

,0,1,2,3
5,-0.539741,0.476985,3.000000,-1.021228
97,-0.774363,0.552936,0.106061,3.000000
102,-0.655054,-0.565230,3.000000,0.959533
305,-2.315555,0.457246,-0.025907,-3.000000
324,0.050188,1.951312,3.000000,0.963301
400,0.146326,0.508391,-0.196713,-3.000000
499,-0.293333,-0.242459,-3.000000,1.918403
523,-3.000000,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.000000,1.369891
808,-0.362528,-3.000000,1.553205,-2.186301


### 排列和随机采样

目的是**打乱顺序**，避免因数据排列带来的问题，比如一份调查问卷数据，由于是按照一个地区一个地区上缴的，因此问卷的顺序也就是各个地区排列的，那么如果此时对数据进行训练/测试分组的话，很容易将某几个区数据分到训练组，而将其他区数据分到测试组，这就无形中引入了不随意的问题，很可能出现在训练集上表现良好的模型在测试集上表现堪忧；

#### 重排数据 -- NumPy.random.permutation

In [124]:
se = Series([1,2,3,4,5])
se[np.random.permutation(5)]

3    4
4    5
2    3
1    2
0    1
dtype: int64

In [123]:
df = DataFrame(np.arange(25).reshape(5,5))
df.ix[np.random.permutation(5)] # 使用df.take一样的效果

/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,0,1,2,3,4
3,15,16,17,18,19
1,5,6,7,8,9
4,20,21,22,23,24
2,10,11,12,13,14
0,0,1,2,3,4


**注意**：索引是保留的；

#### 获取部分重排数据

In [126]:
se[np.random.permutation(5)[:3]] # 获取重排后的数据的前3个

2    3
0    1
4    5
dtype: int64

#### 随机采样 -- 可能重复，使用randint获取随机角标

In [128]:
se[np.random.randint(0, 5, size=3)] # 从0~4之间随机选择3个，此时可能获取到重复的元素

1    2
0    1
3    4
dtype: int64

### 计算 指标/哑 变量

比如性别列，可以转变为两个哑变量，一个为male，取值为true/false，一个为female，取值也是true/false，这样做的好处是true和false是可以进行数学/逻辑运算的，而男女这两个值本身不行，因此对于同一行来说，生成的K个列的总和总是为1的（针对普通情况，每行只属于一个分类）；

In [145]:
df = DataFrame({'Sex':['male','male','female','male','female'],
                'Total Name':['helong','hongkong','anglaboby','jack','england']}, 
               index=['HL','HK','AB','JK','EG'])
df

,Sex,Total Name
HL,male,helong
HK,male,hongkong
AB,female,anglaboby
JK,male,jack
EG,female,england


#### 自己实现

In [149]:
df['Sex_male_my'] = df['Sex'].map(lambda sex:1 if sex=='male' else 0)
df['Sex_female_my'] = df['Sex'].map(lambda sex:1 if sex=='female' else 0)
df

,Sex,Total Name,Sex_male_my,Sex_female_my
HL,male,helong,1,0
HK,male,hongkong,1,0
AB,female,anglaboby,0,1
JK,male,jack,1,0
EG,female,england,0,1


#### pandas.get_dummies

In [150]:
pd.get_dummies(df, columns=['Sex']) # columns指定用于拆分的列

,Total Name,Sex_male_my,Sex_female_my,Sex_female,Sex_male
HL,helong,1,0,0,1
HK,hongkong,1,0,0,1
AB,anglaboby,0,1,1,0
JK,jack,1,0,0,1
EG,england,0,1,1,0


#### 特殊：某行属于多个分类

In [155]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../../../git_data_book/pydata-book/datasets/movielens/movies.dat', 
                       sep='::', header=None, names=mnames)
movies[:10] # 看到此时每个电影的分类是有多个的。。。

/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [164]:
# step1：获取所有分类，要熟练使用python内置的这些方法，split/set.union/*param等
cates = [set(x.split('|')) for x in movies['genres']]
cates = sorted(set.union(*cates))
cates

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [167]:
# step2：循环增加哑变量
for cate in cates:
    movies['|genres_'+cate] = movies['genres'].map(lambda gen:1 if cate in gen.split('|') else 0)
movies[:10]

,movie_id,title,genres,genres_Action,genres_Adventure,genres_Animation,genres_Children's,genres_Comedy,genres_Crime,genres_Documentary,...,|genres_Fantasy,|genres_Film-Noir,|genres_Horror,|genres_Musical,|genres_Mystery,|genres_Romance,|genres_Sci-Fi,|genres_Thriller,|genres_War,|genres_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,7,Sabrina (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
7,8,Tom and Huck (1995),Adventure|Children's,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Sudden Death (1995),Action,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),Action|Adventure|Thriller,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### 对于大数据量时的哑变量问题

此时进行拆分是非常影响性能的，尤其是枚举数比较多的情况下，需要添加很多列数据，对性能和内存都是一种损耗，此时应该结合cut函数进行结合使用，同样的举年龄的例子：

    比如在某份百万级的调查中，有一列表示用户的年龄，因为年龄通常都是一个非常有用的特征，因此此处也想使用，直接使用最方便的方式就是拆分为哑变量，这样可以非常方便的进行统计等操作，但是由于年龄对应的值太多了，1～100都有可能，因此需要配合cut先将年龄分组，再对分组进行拆分，由于组的数量肯定远远小于年龄本身，因此对性能内存等都是巨大的提升；

In [169]:
se = Series([12,4,5,8,22,21,17,45,44,35,60,70,57,63,34,23,70,90])
pd.get_dummies(pd.cut(se, bins=[0,6,18,25,40,1000])) # 先使用cut分组，再使用get_dummies进行拆分

,"(0, 6]","(6, 18]","(18, 25]","(25, 40]","(40, 1000]"
0,0,1,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,1,0,0,0
7,0,0,0,0,1
8,0,0,0,0,1
9,0,0,0,1,0


## 字符串操作

Python能够成为流行的数据处理语言，部分原因是其简单易用的字符串和文本处理功能，大部分文本运算都直接做成了字符串对象的内置方法，对于更为复杂的模式匹配和文本操作，则可能需要用到正则表达式，pandas对此进行了加强，它使你能够**对整组数据应用字符串表达式和正则表达式**，而且能处理烦人的**缺失数据**；

### 字符串对象方法

* count：统计某个字符出现的次数；
* endswith,startswith：返回是否以某个子串结尾/开始；
* join：使用自身连接参数序列中所有的元素；
* index：返回指定元素的角标，不存在抛异常；
* find：返回指定元素的角标，不存在返回-1；
* rfind：同上，反向查找；
* replace：用一个字符串替换另一个；
* strip,lstrip.rstrip：去掉全部/左侧/右侧空格；
* split：切割字符串；
* lower,upper：全部小写/大写；
* ljust,rjust：用空格（或其他字符）填充字符串的空白侧以返回符合最低宽度的字符串；

### 正则表达式

如果打算对许多字符串应用同一条正则表达式，强烈建议通过re.compile创建regex对象，这样将可以节省大量的CPU时间；

* findall,finditer：返回字符串中所有的非重叠匹配模式，findall返回的是一个所有结果的列表，finditer是一个迭代器；
* match：从字符串**起始**位置匹配模式，还可以对模式各部分分组，如果匹配到模式，返回一个匹配项对象，否则None；
* search：全局扫描字符串查找匹配模式；
* split：根据找到的模式拆分字符串；
* sub,subn：将字符串中所有的/n个模式替换为指定表达式（要么是字符串，要么是函数返回值），在替换字符串中可以通过\1,\2等表示各分组项；

### pandas中矢量化的字符串函数 -- 理解成原python字符串函数的矢量化版本 data.str

清理待分析的散乱数据时,常常需要做一些字符串规整化工作，更为复杂的情况是,含有字符串的列有时还含有缺失数据:

In [179]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}
se = Series(data)
se

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [180]:
se.hasnans # 判断是否有缺失值

True

In [181]:
se.map(lambda email:'xxx' if pd.isnull(email) else email)

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  xxx
dtype: object

#### data.str.cat

In [188]:
se.str.cat(sep='><') # 元素级别的连接，可指定连接符

'dave@google.com><rob@gmail.com><steve@gmail.com'

#### data.str.contains

In [190]:
se.str.contains('com') # 矢量化的contains方法，判断每个元素是否有包含参数

Dave     True
Rob      True
Steve    True
Wes       NaN
dtype: object

#### data.str.get

In [195]:
se.str.get(1) # 矢量化的获取某个角标位置的单个字符

Dave       a
Rob        o
Steve      t
Wes      NaN
dtype: object

#### data.str.pad

In [201]:
se.str.pad(fillchar=']', width=50) # 矢量化的ljust/rjust，使用side来指定左右，默认是左

Dave     ]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]dave@google...
Rob      ]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]rob@gmail...
Steve    ]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]steve@gmail...
Wes                                                    NaN
dtype: object

#### data.str.center

In [203]:
se.str.center(fillchar=']', width=30) # 相当于pad(side='both')

Dave     ]]]]]]]dave@google.com]]]]]]]]
Rob      ]]]]]]]]rob@gmail.com]]]]]]]]]
Steve    ]]]]]]]steve@gmail.com]]]]]]]]
Wes                                 NaN
dtype: object

#### data.str.repeat

In [205]:
se.str.repeat(3) # 相当于每个元素*3

Dave     dave@google.comdave@google.comdave@google.com
Rob            rob@gmail.comrob@gmail.comrob@gmail.com
Steve    steve@gmail.comsteve@gmail.comsteve@gmail.com
Wes                                                NaN
dtype: object

#### data.str.slice

In [212]:
se.str.slice(step=2) # 指定起点终点按照一定步长取值

Dave     dv@ogecm
Rob       rbgalcm
Steve    seegalcm
Wes           NaN
dtype: object

#### pandas字符串函数

data.str函数：
* cat：元素级别的连接，可指定连接符
* contains：矢量化的contains方法，判断每个元素是否有包含参数
* count：矢量化版本；
* endswith,startswith：矢量化版本；
* findall：矢量化版本；
* get：矢量化的获取某个角标位置的单个字符
* join：矢量化版本；
* len：矢量化版本；
* lower,upper：矢量化版本；
* match：矢量化版本；
* pad：矢量化的ljust/rjust，使用side来指定左右，默认是左
* center：相当于pad(side='both')
* repeat：相当于每个元素*3
* replace：矢量化版本；
* slice：指定起点终点按照一定步长取值
* split：矢量化版本；
* strip,lstrip,rstrip：矢量化版本；

## 示例：USDA食品数据库

美国农业部(USDA)制作了一份有关食物营养信息的数据库，Ashley Williams(一名来自英国的技术牛人)制作了该数据的JSON版http://ashleyw.co.uk/project/food-nutrient-database ，其中的记录如下所示：
```
{
    "id": 21441,
    "description": "KENTUCKY FRIED CHICKEN, Fried Chicken, EXTRA CRISPY, Wing, meat and",
    "tags": ["KFC"],
    "manufacturer": "Kentucky Fried Chicken",
    "group": "Fast Foods",
    "portions": [
        {
            "amount": 1,
            "unit": "wing, with skin",
            "grams": 68.0
        },
        ...
    ],
    "nutrients": [
        {
            "value": 20.8,
            "units": "g",
            "description": "Protein",
            "group": "Composition"
        },
        ...
    ]
}
```

### 加载展示json数据

pd.read_json('http://ashleyw.co.uk/project/food-nutrient-database')

上述数据暂时无法获取，链接失效了，该单元暂时先不做